In [12]:
import sys
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\change_of_variable')
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\hmc_pytorch')
from change_of_variable_pytorch import * 
from hmc_base_pytorch import *
from hmc_unconstrained_pytorch import *
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
import matplotlib.pyplot as plt

import torch.utils.data
from torch import optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import time
import torchvision


cuda = True
batch_size = 64
epochs = 10
seed = 1
z_dim = 20

# Data preparation
torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_data = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('../data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size, shuffle=True, **kwargs)

def binarization(data):
    s = np.random.uniform(size = data.shape)
    out = np.array(s<data).astype(float)
    return out

result = []
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.view(-1, 784).numpy()
    bi_data = binarization(data)
    d = torch.from_numpy(bi_data)
    result.append(d)
    
result_2d = []
for batch_idx, (data, _) in enumerate(train_loader):
    #data = data.view(-1, 28, 28).numpy()
    #print("data: "+str(data.shape))
    bi_data = binarization(data)
    d = torch.from_numpy(bi_data)
    result_2d.append(d)
    
result_test = []
for batch_idx, (data, _) in enumerate(test_loader):
    data = data.view(-1, 784).numpy()
    bi_data = binarization(data)
    d = torch.from_numpy(bi_data)
    result_test.append(d)

############################################################
def reparameterize(mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.randn_like(std)
    #return eps.mul(std).add_(mu)
    return eps*std+mu

def log_prior(z):
    dim = z.shape[1]
    mean = torch.zeros(dim).cuda()
    cov = torch.eye(dim).cuda()
    m = MultivariateNormal(mean, cov)
    m.requires_grad=True
    return m.log_prob(z)

def multivariate_normal_diagonal_logpdf(mean, cov_diag, x):
    n = x.shape[0] # number of samples
    k = x.shape[1] # dimension
    t1 = -0.5*(x - mean)*(1/cov_diag)*(x-mean)
    t1 = torch.sum(t1, dim=1)
    t2 = torch.ones(n).cuda()*0.5*k*torch.log(torch.tensor([2*math.pi]).cuda()) + 0.5*torch.sum(torch.log(cov_diag),dim=1)
    return t1 - t2
############################################################
"""
class decoder(nn.Module):
    def __init__(self):
        super(decoder, self).__init__()
        self.fc1 = nn.Linear(z_dim, 400)
        self.fc2 = nn.Linear(400, 784)
    # single hidden layer
    def forward(self, x):
        #x = x.view(-1, 784)
        h1 = F.relu(self.fc1(x))
        return F.sigmoid(self.fc2(h1))
"""
class decoder(nn.Module):
    def __init__(self):
        super(decoder, self).__init__()
        self.fc1 = nn.Linear(z_dim, 300)
        self.fc2 = nn.Linear(300, 512)
        self.deconv1 = nn.ConvTranspose2d(32, 32, 5, stride = 2, padding=2)# padding = 2)
        self.deconv2 = nn.ConvTranspose2d(32, 16, 5, stride = 2, padding=2, output_padding=1)#padding = 2, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(16, 1, 5, stride = 2, padding=2, output_padding=1)#padding = 2, output_padding=0)
    # single hidden layer
    def forward(self, x):
        #x = x.view(-1, 784)
        #print("x: "+str(x.shape))
        h1 = F.softplus(self.fc1(x))
        #print("h1: "+str(h1.shape))
        h2 = F.softplus(self.fc2(h1))
        #print("h2: "+str(h2.shape))
        h2 = h2.view(-1, 32, 4, 4)
        #print("h2: "+str(h2.shape))
        h3 = F.softplus(self.deconv1(h2))
        #print("h3: "+str(h3.shape))
        h4 = F.softplus(self.deconv2(h3))
        #print("h4: "+str(h4.shape))
        h5 = self.deconv3(h4)
        #print("h5: "+str(h5.shape))
        #plt.imshow(h5[0].cpu().detach().numpy().reshape(28,28), cmap='gray')
        #plt.show()
        return F.sigmoid(h5.view(-1, 784))    
class q_z0(nn.Module):
    def __init__(self):
        super(q_z0, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, stride = 2, padding = 2 ) # output 12.5
        self.conv2 = nn.Conv2d(16, 32, 5, stride = 2, padding = 2) # 
        self.conv3 = nn.Conv2d(32, 32, 5, stride = 2, padding = 2)
        self.fc1 = nn.Linear(512, 300)
        self.fc21 = nn.Linear(300, z_dim)
        self.fc22 = nn.Linear(300, z_dim)
    def forward(self, x):
        #print("x: "+str(x.shape))
        h1 = F.softplus(self.conv1(x))
        #print(h1.shape)
        h2 = F.softplus(self.conv2(h1))
        #print(h2.shape)
        h3 = F.softplus(self.conv3(h2))
        #print(h3.shape)
        h3 = h3.view(-1, 32*4*4)
        h4 = F.softplus(self.fc1(h3))
        #mu = F.softplus(self.fc21(h4))
        #logvar = F.softplus(self.fc22(h4))
        mu = self.fc21(h4)
        logvar = self.fc22(h4)
        return mu, logvar
class r_v(nn.Module):
    def __init__(self):
        super(r_v, self).__init__()
        self.fc1 = nn.Linear(z_dim + 784, 300)
        self.fc21 = nn.Linear(300, z_dim)
        self.fc22 = nn.Linear(300, z_dim)
    def forward(self, x):
        x = x.view(-1, 784 + z_dim)
        h1 = F.softplus(self.fc1(x))
        logvar = self.fc21(h1)
        mu = self.fc22(h1)
        return mu, logvar

class q_v(nn.Module):
    def __init__(self):
        super(q_v, self).__init__()
        self.fc1 = nn.Linear(784, 300)
        # no need to output mu because the mean of momentum is default 0
        self.fc21 = nn.Linear(300, z_dim)
    def forward(self, x):
        x = x.view(-1, 784)
        h1 = F.softplus(self.fc1(x))
        logvar = self.fc21(h1)
        return logvar
############################################################
decoder = decoder().to(device)
q_z0 = q_z0().to(device)
r_v = r_v().to(device)
q_v = q_v().to(device)

In [13]:
result[0].shape

torch.Size([64, 784])

In [14]:
def ELBO(data, decoder, q_z0, r_v, q_v, log_step, T):#q_z0_mean, q_z0_logvar):
    
    #file1 = open("q_z0_mean"+str(epoch)+".txt","w")
    #file1 = open("q_z0_var"+str(epoch)+".txt","w")
    #file1 = open("z0"+str(epoch)+".txt","w")
    batch_size = data.view(-1, 784).shape[0]
    data = data.to(device)
    
    q_z0_mean, q_z0_logvar = q_z0(data.view(-1, 1, 28, 28))
    # sample z0
    z0 = reparameterize(q_z0_mean, q_z0_logvar)
    
    # compute q(z0|x)
    var_z0 = torch.exp(q_z0_logvar)
    log_q_z0 = multivariate_normal_diagonal_logpdf(q_z0_mean, var_z0, z0)
    
    #print("q z0 mean: "+str(q_z0_mean))
    #print("q z0 var: "+str(var_z0))
    #print("z0: "+str(z0))
    #print("q(z0|x): "+str(log_q_z0))
    #print("np q(z0|x): "+str(np_log_q_z0))
    
    # compute prior
    log_prior_z0 = log_prior(z0)
    
    
    # compute joint
    decoder_output = decoder(z0)
    #print("decoder: "+str(decoder_output))
    log_likelihood = 0. - F.binary_cross_entropy(decoder_output, data.view(-1, 784).float(), size_average=False, reduce=False)
    #print("ll: "+str(log_likelihood))
    log_likelihood = torch.sum(log_likelihood, dim = 1)
    log_joint = log_prior_z0 + log_likelihood
    
    
    # initial L
    #L = log_joint - log_q_z0
    L = -log_q_z0
    #print("initial L: "+str(L))
    #print("log likelihood 0: "+str(torch.mean(log_likelihood)))
    
    for i in range(T):
        # sample v1
        
        logvar_v1 = q_v(data)
        
        #logvar_v1 = torch.zeros([batch_size, z_dim], requires_grad = True).cuda()
        var_v1 = torch.exp(logvar_v1)
        mu_v1 = torch.zeros([logvar_v1.shape[0], logvar_v1.shape[1]],requires_grad = True).cuda()
        v1 = reparameterize(mu_v1, logvar_v1)
        #print("v1: "+str(v1.shape))
        
        # compute q(v1|x) 
        log_q_v1 = multivariate_normal_diagonal_logpdf(mu_v1, var_v1 ,v1)
        #print("log_q_v1: "+str(log_q_v1.shape))
        #print("var_v1: "+str(var_v1))
        
        mass_diag = var_v1
        #print("mass_diag: "+str(mass_diag))
        
        def energy_function(z, cache):
            z = z.view(batch_size, z_dim)
            z_cuda = z.cuda()
            all_log_prior = log_prior(z_cuda)
            #print("all prior: "+str(all_log_prior.shape))
            all_log_prior = torch.sum(all_log_prior)
            #print(all_log_prior.shape)
            decoder_output = decoder(z_cuda)

            all_log_likelihood = 0. - F.binary_cross_entropy(decoder_output, data.view(-1, 784).float(), size_average=False)
            #print("one_log_likelihood: "+str(one_log_likelihood.shape))
            all_log_joint = all_log_prior + all_log_likelihood
            return 0 - all_log_joint
        
        
        init = z0.view(batch_size*z_dim)
        mass_diag = mass_diag.view(mass_diag.shape[0]*mass_diag.shape[1])
        mass_matrix = torch.diag(mass_diag)
        mom = v1.view(v1.shape[0]*v1.shape[1])
        step = torch.exp(log_step)
        #print("init: "+str(init.shape))
        #print("mass: "+str(mass_matrix.shape))
        
        # clip step size
        low = 0.0005
        high = 0.5
        torch.clamp(step, min=low, max=high)
        step = step.repeat(1, batch_size)
        step = step.view(step.shape[1])
        step_cuda = step.cuda()
        #print("log step: "+str(log_step))
        #print("step: "+str(step_cuda))
        
        sampler = IsotropicHmcSampler(energy_function, energy_grad=None, prng=None, mom_resample_coeff=0., dtype=np.float64)
        #pos_samples, mom_samples, ratio = sampler.get_samples(init, 0.0001, 5, 2, mass_matrix, mom = mom)
        pos_samples, mom_samples, ratio = sampler.get_samples(init, step_cuda, 5, 2, mass_matrix, mom = mom)
        
        zt = pos_samples[1].cuda()
        vt = mom_samples[1].cuda()
        
        zt = zt.view(batch_size, z_dim)
        vt = vt.view(batch_size, z_dim)

        # get joint probaility p(x, zt)
        log_prior_zt = log_prior(zt)
        decoder_output_t = decoder(zt)
        #print("log prior: " + str(log_prior_zt.shape))
        #print("decoder: " + str(decoder_output_t.shape))

        log_likelihood_t = 0. - F.binary_cross_entropy(decoder_output_t, data.view(-1, 784).float(), size_average=False, reduce=False)
        log_likelihood_t = torch.sum(log_likelihood_t, dim = 1)
        #print("log ll: " + str(log_likelihood_t.shape))
        
        log_joint_t = log_prior_zt + log_likelihood_t
        #print("log joint t: " + str(log_joint_t.shape))
        
        # get r(vt|x,zt)
        d = data.view(-1, 784)
        #print(d.shape)
        new_data = torch.cat((d.float(), zt), 1) # append data with zt
        #print("new data: "+str(one_new_data.shape))

        mu_vt, logvar_vt = r_v(new_data)
        var_vt = torch.exp(logvar_vt)
        #print("var_vt: "+str(var_vt.shape))

        log_r_vt = multivariate_normal_diagonal_logpdf(mu_vt, var_vt, vt)
        """
        print("log prior t: "+str(torch.mean(log_prior_zt)))
        print("log likelihood t: "+str(torch.mean(log_likelihood_t)))
        print("joint t: "+str(torch.mean(log_joint_t)))
        print("reverse: "+str(torch.mean(log_r_vt)))
        #print(log_joint[j])
        print("forward: "+str(torch.mean(log_q_v1)))
        print("q(z0|x): "+str(torch.mean(log_q_z0)))
        print("========================================`")
        """
        #file1 = open("q_z0_mean"+str(epoch)+".txt","w")
        #file2 = open("q_z0_var"+str(epoch)+".txt","w")
        #file3 = open("z0"+str(epoch)+".txt","w")
        
        # get L for each sample
        #one_log_alpha = one_log_joint_t + one_log_r_vt - log_joint[j] - log_q_v1[j]
        log_alpha = log_joint_t + log_r_vt - log_q_v1
        #one_log_alpha = log_joint[j] #+ one_log_r_vt - log_q_v1[j]

        L = L + log_alpha
        #print("L: "+str(L.shape))
    
    
    return torch.mean(L)

In [15]:
batch_size = 64
z_dim = 20

params1 = list(decoder.parameters())+list(q_z0.parameters())+list(r_v.parameters())+list(q_v.parameters())
optimizer1 = optim.Adam(params1, lr=0.001)#, weight_decay=1e-4)
#optimizer1 = optim.Adam([q_z0_mean], lr=0.005)
#log_step_raw = torch.ones(z_dim, requires_grad=True)
#log_step = -3.0*log_step_raw
log_step=torch.tensor([-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.,-3.], requires_grad=True)
optimizer2 = optim.Adam([log_step], lr=0.001)


for epoch in range(20):
    print("Epoch: "+str(epoch+1))
    file = open("result11_"+str(epoch)+".txt","w")
    file_test = open("result11_test_"+str(epoch)+".txt","w")
    for i in range(len(result)):
        print("++++++++++"+" Epoch: "+str(epoch+1)+" batch: " + str(i) + " ++++++++++")
        data = result[i].float()
        #data_2d = result_2d[i].float()
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss = 0. - ELBO(data, decoder, q_z0, r_v, q_v, log_step, 1)#_mean_cuda, q_z0_logvar_cuda)
        print("ELBO: "+str(0-loss.item()))
        loss.backward()
        #nn.utils.clip_grad_norm_(q_z0.parameters(), 1)
        #print(q_z0.fc1.weight.grad)
        #print(q_z0.fc31.weight.grad)
        #print(q_z0.fc32.weight.grad)
        optimizer1.step()
        optimizer2.step()
        file.write(str(0.-loss.item())+"\n")
    file.close()
    for i in range(len(result_test)):
        print("++++++++++ test batch: " + str(i) + " ++++++++++")
        data = result_test[i].float()
        loss = 0. - ELBO(data, decoder, q_z0, r_v, q_v, log_step, 1)
        print("ELBO: "+str(0-loss.item()))
        file_test.write(str(0.-loss.item())+"\n")
        
    file_test.close()
    
    sample = torch.randn(64, 20).to(device)
    sample = decoder(sample).cpu()
    save_image(sample.view(64, 1, 28, 28), 'sample11_' + str(epoch) + '.png')

Epoch: 1
++++++++++ Epoch: 1 batch: 0 ++++++++++
ELBO: -528.7647705078125
++++++++++ Epoch: 1 batch: 1 ++++++++++
ELBO: -501.4366455078125
++++++++++ Epoch: 1 batch: 2 ++++++++++
ELBO: -479.5432434082031
++++++++++ Epoch: 1 batch: 3 ++++++++++
ELBO: -456.335205078125
++++++++++ Epoch: 1 batch: 4 ++++++++++
ELBO: -432.7338562011719
++++++++++ Epoch: 1 batch: 5 ++++++++++
ELBO: -407.26123046875
++++++++++ Epoch: 1 batch: 6 ++++++++++
ELBO: -384.6111755371094
++++++++++ Epoch: 1 batch: 7 ++++++++++
ELBO: -370.3451843261719
++++++++++ Epoch: 1 batch: 8 ++++++++++
ELBO: -372.5983581542969
++++++++++ Epoch: 1 batch: 9 ++++++++++
ELBO: -377.7830505371094
++++++++++ Epoch: 1 batch: 10 ++++++++++
ELBO: -375.86328125
++++++++++ Epoch: 1 batch: 11 ++++++++++
ELBO: -357.2648620605469
++++++++++ Epoch: 1 batch: 12 ++++++++++
ELBO: -335.0726623535156
++++++++++ Epoch: 1 batch: 13 ++++++++++
ELBO: -343.57720947265625
++++++++++ Epoch: 1 batch: 14 ++++++++++
ELBO: -332.6886901855469
++++++++++ Epoch: 

ELBO: -206.799072265625
++++++++++ Epoch: 1 batch: 124 ++++++++++
ELBO: -206.11856079101562
++++++++++ Epoch: 1 batch: 125 ++++++++++
ELBO: -213.14305114746094
++++++++++ Epoch: 1 batch: 126 ++++++++++
ELBO: -217.01792907714844
++++++++++ Epoch: 1 batch: 127 ++++++++++
ELBO: -211.77964782714844
++++++++++ Epoch: 1 batch: 128 ++++++++++
ELBO: -216.2801971435547
++++++++++ Epoch: 1 batch: 129 ++++++++++
ELBO: -214.61151123046875
++++++++++ Epoch: 1 batch: 130 ++++++++++
ELBO: -216.6817626953125
++++++++++ Epoch: 1 batch: 131 ++++++++++
ELBO: -213.85311889648438
++++++++++ Epoch: 1 batch: 132 ++++++++++
ELBO: -204.9689178466797
++++++++++ Epoch: 1 batch: 133 ++++++++++
ELBO: -220.35377502441406
++++++++++ Epoch: 1 batch: 134 ++++++++++
ELBO: -209.6273193359375
++++++++++ Epoch: 1 batch: 135 ++++++++++
ELBO: -206.49703979492188
++++++++++ Epoch: 1 batch: 136 ++++++++++
ELBO: -202.90699768066406
++++++++++ Epoch: 1 batch: 137 ++++++++++
ELBO: -212.48191833496094
++++++++++ Epoch: 1 batch: 1

ELBO: -222.50741577148438
++++++++++ Epoch: 1 batch: 246 ++++++++++
ELBO: -212.6266632080078
++++++++++ Epoch: 1 batch: 247 ++++++++++
ELBO: -211.4329071044922
++++++++++ Epoch: 1 batch: 248 ++++++++++
ELBO: -201.28826904296875
++++++++++ Epoch: 1 batch: 249 ++++++++++
ELBO: -213.3784942626953
++++++++++ Epoch: 1 batch: 250 ++++++++++
ELBO: -207.75267028808594
++++++++++ Epoch: 1 batch: 251 ++++++++++
ELBO: -224.57565307617188
++++++++++ Epoch: 1 batch: 252 ++++++++++
ELBO: -209.7626190185547
++++++++++ Epoch: 1 batch: 253 ++++++++++
ELBO: -204.4600372314453
++++++++++ Epoch: 1 batch: 254 ++++++++++
ELBO: -207.60467529296875
++++++++++ Epoch: 1 batch: 255 ++++++++++
ELBO: -194.7031707763672
++++++++++ Epoch: 1 batch: 256 ++++++++++
ELBO: -201.02557373046875
++++++++++ Epoch: 1 batch: 257 ++++++++++
ELBO: -219.0824737548828
++++++++++ Epoch: 1 batch: 258 ++++++++++
ELBO: -215.30140686035156
++++++++++ Epoch: 1 batch: 259 ++++++++++
ELBO: -205.67724609375
++++++++++ Epoch: 1 batch: 260 +

ELBO: -214.91180419921875
++++++++++ Epoch: 1 batch: 368 ++++++++++
ELBO: -200.9508819580078
++++++++++ Epoch: 1 batch: 369 ++++++++++
ELBO: -198.33291625976562
++++++++++ Epoch: 1 batch: 370 ++++++++++
ELBO: -206.0774383544922
++++++++++ Epoch: 1 batch: 371 ++++++++++
ELBO: -213.06768798828125
++++++++++ Epoch: 1 batch: 372 ++++++++++
ELBO: -209.42019653320312
++++++++++ Epoch: 1 batch: 373 ++++++++++
ELBO: -199.52255249023438
++++++++++ Epoch: 1 batch: 374 ++++++++++
ELBO: -204.33168029785156
++++++++++ Epoch: 1 batch: 375 ++++++++++
ELBO: -205.5135040283203
++++++++++ Epoch: 1 batch: 376 ++++++++++
ELBO: -213.65179443359375
++++++++++ Epoch: 1 batch: 377 ++++++++++
ELBO: -203.70486450195312
++++++++++ Epoch: 1 batch: 378 ++++++++++
ELBO: -213.28919982910156
++++++++++ Epoch: 1 batch: 379 ++++++++++
ELBO: -213.16647338867188
++++++++++ Epoch: 1 batch: 380 ++++++++++
ELBO: -193.556396484375
++++++++++ Epoch: 1 batch: 381 ++++++++++
ELBO: -200.07737731933594
++++++++++ Epoch: 1 batch: 

ELBO: -211.8597412109375
++++++++++ Epoch: 1 batch: 490 ++++++++++
ELBO: -202.93544006347656
++++++++++ Epoch: 1 batch: 491 ++++++++++
ELBO: -202.17759704589844
++++++++++ Epoch: 1 batch: 492 ++++++++++
ELBO: -208.48187255859375
++++++++++ Epoch: 1 batch: 493 ++++++++++
ELBO: -209.18658447265625
++++++++++ Epoch: 1 batch: 494 ++++++++++
ELBO: -206.16412353515625
++++++++++ Epoch: 1 batch: 495 ++++++++++
ELBO: -198.65115356445312
++++++++++ Epoch: 1 batch: 496 ++++++++++
ELBO: -202.24429321289062
++++++++++ Epoch: 1 batch: 497 ++++++++++
ELBO: -206.9957275390625
++++++++++ Epoch: 1 batch: 498 ++++++++++
ELBO: -197.03158569335938
++++++++++ Epoch: 1 batch: 499 ++++++++++
ELBO: -207.5584259033203
++++++++++ Epoch: 1 batch: 500 ++++++++++
ELBO: -205.1439208984375
++++++++++ Epoch: 1 batch: 501 ++++++++++
ELBO: -204.90914916992188
++++++++++ Epoch: 1 batch: 502 ++++++++++
ELBO: -212.09547424316406
++++++++++ Epoch: 1 batch: 503 ++++++++++
ELBO: -202.4664764404297
++++++++++ Epoch: 1 batch: 

...

ELBO: -98.14300537109375
++++++++++ Epoch: 7 batch: 321 ++++++++++
ELBO: -101.52933502197266
++++++++++ Epoch: 7 batch: 322 ++++++++++
ELBO: -103.2413330078125
++++++++++ Epoch: 7 batch: 323 ++++++++++
ELBO: -99.6992416381836
++++++++++ Epoch: 7 batch: 324 ++++++++++
ELBO: -102.19552612304688
++++++++++ Epoch: 7 batch: 325 ++++++++++
ELBO: -102.89701843261719
++++++++++ Epoch: 7 batch: 326 ++++++++++
ELBO: -98.62065887451172
++++++++++ Epoch: 7 batch: 327 ++++++++++
ELBO: -107.7331314086914
++++++++++ Epoch: 7 batch: 328 ++++++++++
ELBO: -101.45800018310547
++++++++++ Epoch: 7 batch: 329 ++++++++++
ELBO: -106.5079116821289
++++++++++ Epoch: 7 batch: 330 ++++++++++
ELBO: -103.30206298828125
++++++++++ Epoch: 7 batch: 331 ++++++++++
ELBO: -94.30947875976562
++++++++++ Epoch: 7 batch: 332 ++++++++++
ELBO: -103.24405670166016
++++++++++ Epoch: 7 batch: 333 ++++++++++
ELBO: -101.84549713134766
++++++++++ Epoch: 7 batch: 334 ++++++++++
ELBO: -100.58020782470703
++++++++++ Epoch: 7 batch: 335

ELBO: -102.11701202392578
++++++++++ Epoch: 7 batch: 443 ++++++++++
ELBO: -101.19441986083984
++++++++++ Epoch: 7 batch: 444 ++++++++++
ELBO: -101.13685607910156
++++++++++ Epoch: 7 batch: 445 ++++++++++
ELBO: -103.24530792236328
++++++++++ Epoch: 7 batch: 446 ++++++++++
ELBO: -108.45893859863281
++++++++++ Epoch: 7 batch: 447 ++++++++++
ELBO: -99.19454956054688
++++++++++ Epoch: 7 batch: 448 ++++++++++
ELBO: -97.10563659667969
++++++++++ Epoch: 7 batch: 449 ++++++++++
ELBO: -101.63389587402344
++++++++++ Epoch: 7 batch: 450 ++++++++++
ELBO: -99.84341430664062
++++++++++ Epoch: 7 batch: 451 ++++++++++
ELBO: -106.5803451538086
++++++++++ Epoch: 7 batch: 452 ++++++++++
ELBO: -102.77130889892578
++++++++++ Epoch: 7 batch: 453 ++++++++++
ELBO: -103.0474624633789
++++++++++ Epoch: 7 batch: 454 ++++++++++
ELBO: -104.94856262207031
++++++++++ Epoch: 7 batch: 455 ++++++++++
ELBO: -106.49566650390625
++++++++++ Epoch: 7 batch: 456 ++++++++++
ELBO: -93.87635040283203
++++++++++ Epoch: 7 batch: 4

ELBO: -107.60650634765625
++++++++++ Epoch: 7 batch: 565 ++++++++++
ELBO: -103.4303970336914
++++++++++ Epoch: 7 batch: 566 ++++++++++
ELBO: -97.54017639160156
++++++++++ Epoch: 7 batch: 567 ++++++++++
ELBO: -101.09818267822266
++++++++++ Epoch: 7 batch: 568 ++++++++++
ELBO: -100.89527893066406
++++++++++ Epoch: 7 batch: 569 ++++++++++
ELBO: -95.84870910644531
++++++++++ Epoch: 7 batch: 570 ++++++++++
ELBO: -102.87376403808594
++++++++++ Epoch: 7 batch: 571 ++++++++++
ELBO: -105.64742279052734
++++++++++ Epoch: 7 batch: 572 ++++++++++
ELBO: -106.8223648071289
++++++++++ Epoch: 7 batch: 573 ++++++++++
ELBO: -102.55445098876953
++++++++++ Epoch: 7 batch: 574 ++++++++++
ELBO: -100.90423583984375
++++++++++ Epoch: 7 batch: 575 ++++++++++
ELBO: -111.27681732177734
++++++++++ Epoch: 7 batch: 576 ++++++++++
ELBO: -99.98786163330078
++++++++++ Epoch: 7 batch: 577 ++++++++++
ELBO: -102.26004791259766
++++++++++ Epoch: 7 batch: 578 ++++++++++
ELBO: -103.2547836303711
++++++++++ Epoch: 7 batch: 5

ELBO: -93.6918716430664
++++++++++ Epoch: 7 batch: 687 ++++++++++
ELBO: -102.74116516113281
++++++++++ Epoch: 7 batch: 688 ++++++++++
ELBO: -102.05834197998047
++++++++++ Epoch: 7 batch: 689 ++++++++++
ELBO: -99.44837188720703
++++++++++ Epoch: 7 batch: 690 ++++++++++
ELBO: -99.35879516601562
++++++++++ Epoch: 7 batch: 691 ++++++++++
ELBO: -101.88371276855469
++++++++++ Epoch: 7 batch: 692 ++++++++++
ELBO: -101.96678161621094
++++++++++ Epoch: 7 batch: 693 ++++++++++
ELBO: -100.77777862548828
++++++++++ Epoch: 7 batch: 694 ++++++++++
ELBO: -104.7559814453125
++++++++++ Epoch: 7 batch: 695 ++++++++++
ELBO: -95.73090362548828
++++++++++ Epoch: 7 batch: 696 ++++++++++
ELBO: -103.36056518554688
++++++++++ Epoch: 7 batch: 697 ++++++++++
ELBO: -105.96366119384766
++++++++++ Epoch: 7 batch: 698 ++++++++++


KeyboardInterrupt: 

In [ ]:
dt = torch.tensor([1., 2.])
tmp = torch.tensor([2., 4.])
x = dt*tmp
x.shape

In [39]:
x = torch.tensor([[1., 2.],[0., 0.]]).view(-1, 4)
x

tensor([[ 1.,  2.,  0.,  0.]])

In [40]:
x.view(2,2)

tensor([[ 1.,  2.],
        [ 0.,  0.]])